In [1]:
import matplotlib
matplotlib.use('Agg')
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
from stagpy.stagyydata import StagyyData
from stagpy import field as sp_field

# --- USER INPUT ---
plot_mode = "time"       # Set to "time" or "snapshot"
target_time_Gyr = 1.9    # Used if plot_mode is "time"
target_snapshot = 6000   # Used if plot_mode is "snapshot"

field_to_plot = "eta"    # "T", "eta", or "basalt"

# --- CONFIGURATION ---
field_limits = {
    "T": {"vmin": 300, "vmax": 4000},
    "basalt": {"vmin": 0.0, "vmax": 1.0},
    "eta": {"vmin": 1e18, "vmax": 1e25}
}
field_labels = {"T": "Temperature", "eta": "Viscosity", "basalt": "Basalt Fraction"}

# Data path
data_path = Path("/media/aritro/f522493b-003a-404d-a839-3e0925c674b6/Aritro/StagYY/runs/venus_02/archive/")
sdat = StagyyData(data_path)

# Extract folder name for naming (gets 'venus_01' from the path)
folder_name = data_path.parent.name 

SEC_PER_GYR = 1e9 * 365.25 * 24 * 3600

# --- 1. SELECTION LOGIC ---
snap_number = None
actual_time_Gyr = None

if plot_mode == "time":
    print(f"Searching for snapshot closest to {target_time_Gyr} Gyr...")
    snap_indices = []
    snap_times_seconds = []

    for snap in sdat.snaps:
        try:
            t = snap.time
            if t is None: t = snap.timeinfo["time"]
            snap_indices.append(snap.isnap)
            snap_times_seconds.append(t)
        except:
            continue
    
    snap_times_seconds = np.array(snap_times_seconds)
    snap_indices = np.array(snap_indices)

    if len(snap_times_seconds) == 0:
        raise ValueError("No valid timestamps found. Check your _time.dat file.")

    target_seconds = target_time_Gyr * SEC_PER_GYR
    idx = np.abs(snap_times_seconds - target_seconds).argmin()
    snap_number = int(snap_indices[idx])
    actual_time_Gyr = snap_times_seconds[idx] / SEC_PER_GYR
else:
    # Snapshot Mode
    try:
        snap_number = target_snapshot
        snapshot = sdat.snaps[snap_number]
        t = snapshot.time
        if t is None: t = snapshot.timeinfo["time"]
        actual_time_Gyr = t / SEC_PER_GYR
        print(f"Snapshot Mode: Selected Snap {snap_number}")
    except Exception as e:
        print(f"Error finding snapshot {target_snapshot}: {e}")
        snap_number = None

# --- 2. GENERATE THE PLOT ---
if snap_number is not None:
    try:
        snapshot = sdat.snaps[snap_number]
        lims = field_limits.get(field_to_plot)
        
        if field_to_plot == "eta":
            norm = colors.LogNorm(vmin=lims["vmin"], vmax=lims["vmax"])
            fig, ax, mesh, cbar = sp_field.plot_scalar(snapshot, field_to_plot, norm=norm)
        else:
            fig, ax, mesh, cbar = sp_field.plot_scalar(snapshot, field_to_plot, 
                                                     vmin=lims["vmin"], vmax=lims["vmax"])
        
        # Colorbar Labeling
        unit = snapshot.fields[field_to_plot].meta.dim
        display_name = field_labels.get(field_to_plot, field_to_plot)
        cbar.set_label(f"{display_name} [{unit}]")
        
        # TIME LABEL ON PLOT (Kept same as before)
        ax.text(0.5, 0.5, f"{actual_time_Gyr:.3f} Gyr", 
                transform=ax.transAxes, ha="center", va="center", 
                fontsize=20, color="black", 
                bbox=dict(facecolor="white", alpha=0.7, edgecolor="none"))
        
        fig.set_size_inches(10, 6)
        plt.tight_layout()
        
        # NEW NAMING SCHEME: [folder]_[field]_snap-[number]_[time]-Gyr.png
        save_name = f"{folder_name}_{field_to_plot}_snap-{snap_number}_{actual_time_Gyr:.1f}-Gyr.png"
        fig.savefig(save_name, dpi=400)
        plt.close(fig)
        
        print(f"Successfully saved: {save_name}")

    except Exception as e:
        print(f"An error occurred during plotting: {e}")

Searching for snapshot closest to 1.9 Gyr...
Successfully saved: venus_02_eta_snap-6515_1.9-Gyr.png


In [ ]:
# This will list the indices of all snapshots found in the folder
snap_indices = [snap.isnap for snap in sdat.snaps]
print(f"Number of snapshots: {len(snap_indices)}")
print(f"Snapshot indices found: {snap_indices}")